# Sprint 09. Tasca 02. Aprenentatge supervisat. Regresions
## By José Manuel Castaño

## - Exercici 1

Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
airlines = pd.read_csv('DelayedFlights.csv', index_col=0, engine='python')

In [3]:
airlines

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,2008,12,13,6,1250.0,1220,1617.0,1552,DL,1621,N938DL,147.0,152.0,120.0,25.0,30.0,MSP,ATL,906,9.0,18.0,0,N,0,3.0,0.0,0.0,0.0,22.0
7009717,2008,12,13,6,657.0,600,904.0,749,DL,1631,N3743H,127.0,109.0,78.0,75.0,57.0,RIC,ATL,481,15.0,34.0,0,N,0,0.0,57.0,18.0,0.0,0.0
7009718,2008,12,13,6,1007.0,847,1149.0,1010,DL,1631,N909DA,162.0,143.0,122.0,99.0,80.0,ATL,IAH,689,8.0,32.0,0,N,0,1.0,0.0,19.0,0.0,79.0
7009726,2008,12,13,6,1251.0,1240,1446.0,1437,DL,1639,N646DL,115.0,117.0,89.0,9.0,11.0,IAD,ATL,533,13.0,13.0,0,N,0,NaN,NaN,NaN,NaN,NaN


La descripció dels diferents camps, l'estudi de la distribució de les variables, els Nans, etc, ja ho hem fet en exercicis anteriors. 

Elimimem del data set els vols cancel.lats i els desviats ja que no ens serveixen per estudiar els retards. A continuació, eliminem els camps **Cancelled, CancellationCode, Diverted**

In [3]:
airlines=airlines[airlines['Cancelled']==0]
airlines=airlines[airlines['Diverted'] ==0]
airlines.drop(['Cancelled', 'CancellationCode', 'Diverted'], axis=1, inplace=True)

Observem que ActualElapsedTime = AirTime + TaxiIn + TaxiOut

In [5]:
sum(airlines.AirTime + airlines.TaxiIn + airlines.TaxiOut - airlines.ActualElapsedTime)

0.0

In [6]:
#ArrDelay mínim a partir del qual s'informen els camps de causa del retards
airlines[airlines.CarrierDelay.notnull()]['ArrDelay'].min()

15.0

In [7]:
#Comprobem que ArrDelay = CarrierDelay + WeatherDelay + NASDelay + SecurityDelay + SecurityDelay + LateAircraftDelay
df=airlines[airlines.CarrierDelay.notnull()][['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','ArrDelay']]
df['ArrDelay']= df.ArrDelay * -1
df.sum(axis=1).sum()

0.0

Observem que els vols que tenen un retard superior a 15min tenen informats els camps CarrierDelay, WeatherDelay, NASDelay, SecurityDelay i LateAircraftDelay. Igualment, els vols que tenen informats aquests camps, la seva suma és ArrDelay:

ArrDelay = CarrierDelay + WeatherDelay + NASDelay + SecurityDelay + SecurityDelay + LateAircraftDelay

És a dir, que en els vols que tinguin informades les causes del retad (1.247.488 vols), el target ArrDelay quedaria totalment explicat per la seva suma, amb correlació = 1.

### Eliminació de variables

En aquest primer model, eliminem algunes variables per diferents motius:

- **CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, SecurityDelay, LateAircraftDelay** ja que expliquen la causa del retard per als vols amb retard >15' de forma exacta. Te la pinta que han omplert quan ja es coneix el retard i s'han quadrat exactament amb el retard.
- **ActualElapsedTime** ja que ActualElapsedTime = AirTime + TaxiIn + TaxiOut
- **FlightNum, TailNum** perquè entenem que no afecten al target.
- **Year** perquè sempre és 2008
- **DepTime, ArrTime, CRSArrTime, AirTime, CRSElapsedTime**. Ens quedem CRSDepTime com a variable d'horari

En funció dels resultats dels models, potser tornariem a agafar algunes de les variables inicialment rebutjades


In [4]:
airlines1=airlines.drop(['CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'SecurityDelay', 'LateAircraftDelay','ActualElapsedTime','FlightNum','TailNum', 'Year','DepTime', 'ArrTime', 'CRSArrTime', 'AirTime', 'CRSElapsedTime'], axis=1)
airlines1

,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,3,4,1955,WN,-14.0,8.0,IAD,TPA,810,4.0,8.0
1,1,3,4,735,WN,2.0,19.0,IAD,TPA,810,5.0,10.0
2,1,3,4,620,WN,14.0,8.0,IND,BWI,515,3.0,17.0
4,1,3,4,1755,WN,34.0,34.0,IND,BWI,515,3.0,10.0
5,1,3,4,1915,WN,11.0,25.0,IND,JAX,688,4.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1220,DL,25.0,30.0,MSP,ATL,906,9.0,18.0
7009717,12,13,6,600,DL,75.0,57.0,RIC,ATL,481,15.0,34.0
7009718,12,13,6,847,DL,99.0,80.0,ATL,IAH,689,8.0,32.0
7009726,12,13,6,1240,DL,9.0,11.0,IAD,ATL,533,13.0,13.0


Per tal de seleccionar les variables a utilitzar en els diferents models, fem una correlació amb el target

In [9]:
airlines1.corr()['ArrDelay']

Month        -0.000897
DayofMonth    0.004129
DayOfWeek     0.006123
CRSDepTime    0.044447
ArrDelay      1.000000
DepDelay      0.952927
Distance     -0.029853
TaxiIn        0.156277
TaxiOut       0.286484
Name: ArrDelay, dtype: float64

### Selecció de variables
Seleccionem les 3 variables que tenen una correlació més alta amb el target: **DepDelay, TaxiOut, TaxiIn**

Agafem una mostra del 10% i la dividim en el train i test


In [5]:
airlines1_sample= airlines1.sample(frac=0.1, random_state=0)
y=airlines1_sample['ArrDelay'].to_numpy()
x=airlines1_sample[['DepDelay', 'TaxiOut', 'TaxiIn']].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

Farem 3 models de regressió: **Linear Regression, Decision Tree Regression i GradientBoostingRegressor**

### Model 1: Linear Regression

In [6]:
lr=LinearRegression().fit(X_train, y_train)

Calculem les mètriques R2 i MSE (Mean square error)

In [7]:
R2_Train = r2_score(y_train,lr.predict(X_train)).round(4)
R2_Test= r2_score(y_test,lr.predict(X_test)).round(4)
MSE_Train= mean_squared_error(y_train,lr.predict(X_train)).round(4)
MSE_Test= mean_squared_error(y_test,lr.predict(X_test)).round(4)

In [8]:
print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 0.9599       R2_Test: 0.957
MSE_Train: 133.4369    MSE_Test: 132.8652


Veiem que R2 i MSE són similars en el dataset de train i el de test

Creem un DF amb les mètriques dels models per tal de facilitar la comparació posterior

In [9]:
data={'Model':'LinearRegression', 'R2_Train':R2_Train , 'R2_Test': R2_Test, 'MSE_Train':MSE_Train, 'MSE_Test':MSE_Test}
metrics = pd.DataFrame(data, index=[0])
metrics

,Model,R2_Train,R2_Test,MSE_Train,MSE_Test
0,LinearRegression,0.9599,0.957,133.4369,132.8652


### Model 2: Decision Tree Regression

In [10]:
dt = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)

Calculem les mètriques R2 i MSE (Mean square error)

In [11]:
R2_Train = r2_score(y_train, dt.predict(X_train)).round(4)
R2_Test= r2_score(y_test, dt.predict(X_test)).round(4)
MSE_Train= mean_squared_error(y_train, dt.predict(X_train)).round(4)
MSE_Test= mean_squared_error(y_test, dt.predict(X_test)).round(4)

In [12]:
print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 0.9795       R2_Test: 0.9323
MSE_Train: 68.1575    MSE_Test: 209.1717


Veiem que R2 són bastant similars en el dataset de train i de test, en canvi el MSE és 3 vegades superior en el dataset de test

In [13]:
#Afegim les mètriques al df de metrics
data={'Model':'DecisionTreeRegressor', 'R2_Train':R2_Train , 'R2_Test': R2_Test, 'MSE_Train':MSE_Train, 'MSE_Test':MSE_Test}
metrics= metrics.append(data, ignore_index=True)

### Model 3: GradientBoostingRegressor

In [14]:
gbr = GradientBoostingRegressor(random_state=0).fit(X_train, y_train)

Calculem les mètriques R2 i MSE (Mean square error)

In [15]:
R2_Train = r2_score(y_train, gbr.predict(X_train)).round(4)
R2_Test= r2_score(y_test, gbr.predict(X_test)).round(4)
MSE_Train= mean_squared_error(y_train, gbr.predict(X_train)).round(4)
MSE_Test= mean_squared_error(y_test, gbr.predict(X_test)).round(4)

In [16]:
print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 0.9614       R2_Test: 0.9578
MSE_Train: 128.4322    MSE_Test: 130.3502


Veiem que R2 i MSE són similars en el dataset de train i el de test

In [17]:
#Afegim les mètriques al df de metrics
data={'Model':'GradientBoostingRegressor', 'R2_Train':R2_Train , 'R2_Test': R2_Test, 'MSE_Train':MSE_Train, 'MSE_Test':MSE_Test}
metrics = metrics.append(data, ignore_index=True)

## - Exercici 2

Compara’ls en base al MSE i al R2

Hem creat la matriu metrics amb R2 i MSE de train i test dels 3 models

In [18]:
metrics

,Model,R2_Train,R2_Test,MSE_Train,MSE_Test
0,LinearRegression,0.9599,0.9570,133.4369,132.8652
1,DecisionTreeRegressor,0.9795,0.9323,68.1575,209.1717
2,GradientBoostingRegressor,0.9614,0.9578,128.4322,130.3502


### Comparativa dels Models
- La millor R2 de train la té el model DecissionTreeRegressor amb **0.9795** tot i que no hi ha tanta diferència amb els altres models.
- La millor R2 de test la té el model GradientBoostingRegressor amb **0.9578** tot i que no hi ha tanta diferència amb els altres models.
- La millor MSE de train la té el model DecissionTreeRegressor amb **68.157** que és la meitat dels altres models
- La millor MSE de test la té el model GradientBoostingRegressor amb **130.350** que és molt similar al LinearRegression   
Si hagués d'escollir un únic model i a la vista del R2 i MSE d'aquests 3 models i sense cap altre informació, agafaria **DecissionTreeRegressor** perquè presenta les millors mètriques en train.

## - Exercici 3

Entrena’ls utilitzant els diferents paràmetres que admeten.

Per determinar els millors paràmetres els entrenarem utilitzant GridSearch

### 1.- Linear Regression

In [19]:
#Veiem els paràmetres amb els que hem entrenat la Linear Regression
lr.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': 'deprecated',
 'positive': False}

In [29]:
# GridSearch
parameters ={'copy_X': [True, False], 'fit_intercept': [True, False]}
gs = GridSearchCV(estimator = LinearRegression(), param_grid=parameters, scoring = 'r2')
gs.fit(X_train, y_train)

print("Best parameters:", gs.best_params_)

Best parameters: {'copy_X': True, 'fit_intercept': True}


Els millors paràmetres que proposa són els mateixos que haviem utilitzat. Igualment, farem la prova de canviar-los i comparar

In [30]:
lr1=LinearRegression(copy_X = False, fit_intercept = False).fit(X_train, y_train)
print('r2:', r2_score(y_train, lr1.predict(X_train)).round(4))
print('MSE: ', mean_squared_error(y_train, lr1.predict(X_train)).round(4))

r2: 0.9303
MSE:  231.7137


Veiem que **el model empitjora**: r2 passa de 0.959 a 0.9303 i el MSE passa de 133.436 a 231.713. Per tant, ens quedem amb els paràmetres inicials

###  2: Decision Tree Regression

In [30]:
#Veiem els paràmetres amb els que hem entrenat la Decision Tree Regression
dt.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 0,
 'splitter': 'best'}

In [31]:
# GridSearch del DecisionTreeRegressor
parameters ={'criterion':["squared_error", "friedman_mse"], 'max_features':["auto", None], 'max_depth': [None, 3],
             'splitter': ['best', 'random']}
gs = GridSearchCV(estimator = DecisionTreeRegressor(), param_grid=parameters, scoring = 'r2')
gs.fit(X_train, y_train)

print("Best parameters:", gs.best_params_)

Best parameters: {'criterion': 'squared_error', 'max_depth': None, 'max_features': None, 'splitter': 'random'}


Entrenem el DecisionTreeRegressor amb els nous paràmetres proposats

In [32]:
dt1 = DecisionTreeRegressor(random_state=0, criterion= 'friedman_mse', splitter= 'random').fit(X_train, y_train)
print('r2: ', r2_score(y_train, dt1.predict(X_train)).round(4))
print('MSE: ', mean_squared_error(y_train, dt1.predict(X_train)).round(4))

r2:  0.9795
MSE:  68.1575


Veiem que el model **no millora**, queda com estava. De fet era el model ja presentava els millors paràmetres

### 3: GradientBoostingRegressor

In [33]:
#Veiem els paràmetres amb els que hem entrenat la Decision Tree Regression
gbr.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 0,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [34]:
#GridSearch del GradientBosstingRegressor
parameters= {'learning_rate': [0.1, 0.05], 'max_depth': [3, 5], 'n_estimators': [100, 250, 500]}
gs = GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = parameters, scoring = 'r2')
gs.fit(X_train,y_train)

print("Best parameters:", gs.best_params_)

Best parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500}


Entrenem el GradientBoostingRegressor amb els nous paràmetres proposats

In [41]:
gbr1 = GradientBoostingRegressor(random_state=0, learning_rate= 0.05, n_estimators= 500).fit(X_train, y_train)
print('r2: ', r2_score(y_train, gbr1.predict(X_train)).round(4))
print('MSE: ', mean_squared_error(y_train, gbr.predict(X_train)).round(4))

r2:  0.9618
MSE:  128.4322


Veiem que amb els nous paràmetres el model de GradientBoostingRegressor **no millora**

### Millores dels models:
Després de fer un GridSearch i d'ajustar els tres models amb els paràmetres proposats, tenim:
- **LinearRegression** no millora, per tant mantenim els paràmetres per defecte
- **DecisionTreeRegression** ens proposa altres paràmetres pero al aplicar-los no millora les mètriques que teniem amb el paràmetres per defecte
- **GradientBoostingRegressor** ens proposa altres paràmetres pero al aplicar-los no millora les mètriques que teniem amb el paràmetres per defecte   
Els tres models tenien R2 superiors al 95% i per tant és dificil aconseguir millores sensibles per la via de modificar paràmetres dels models

## - Exercici 4

Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna)

Anteriorment haviem creat els 3 models aplicant train/test sobre una mostra del 10% del total del dataset.   
Ara, obtindrem les mètriques aplicant el cross validation (de 10) partint de la mateixa mostra del 10% del total del dataset

### 1.- Linear Regression

In [20]:
r2 = cross_val_score(LinearRegression(), x, y, cv = 10, scoring = 'r2')
mse = cross_val_score(LinearRegression(), x, y, cv = 10, scoring = 'neg_mean_squared_error')
r2 =np.mean(r2).round(4)
mse = abs(np.mean(mse)).round(4)
print('r2: {0}\nmse: {1}'.format(r2, mse))

r2: 0.959
mse: 133.2762


Creem un DF amb les mètriques dels models de cross validation per tal de facilitar la comparació posterior

In [21]:
data= {'Model':'LinearRegression', 'R2_CV':r2, 'MSE_CV':mse}
metrics_cv= pd.DataFrame(data, index=[0])
metrics_cv

,Model,R2_CV,MSE_CV
0,LinearRegression,0.959,133.2762


### 2.- Decision Tree Regression

In [22]:
r2 = cross_val_score(DecisionTreeRegressor(), x, y, cv = 10, scoring = 'r2')
mse = cross_val_score(DecisionTreeRegressor(), x, y, cv = 10, scoring = 'neg_mean_squared_error')
r2 =np.mean(r2).round(4)
mse = abs(np.mean(mse)).round(4)
print('r2: {0}\nmse: {1}'.format(r2, mse))

r2: 0.9385
mse: 200.4229


In [23]:
#Afegim les dades al DF de metrics_cv
data= {'Model':'DecisionTreeRegressor', 'R2_CV':r2, 'MSE_CV':mse}
metrics_cv= metrics_cv.append(data, ignore_index=True)

### 3.- GradientBoostingRegressor

In [24]:
r2 = cross_val_score(GradientBoostingRegressor(), x, y, cv = 10, scoring = 'r2')
mse = cross_val_score(GradientBoostingRegressor(), x, y, cv = 10, scoring = 'neg_mean_squared_error')
r2 =np.mean(r2).round(4)
mse = abs(np.mean(mse)).round(4)
print('r2: {0}\nmse: {1}'.format(r2, mse))

r2: 0.9597
mse: 130.9626


In [25]:
#Afegim les dades al DF de metrics_cv
data= {'Model':'GradientBoostingRegressor', 'R2_CV':r2, 'MSE_CV':mse}
metrics_cv = metrics_cv.append(data, ignore_index=True)
metrics_cv

,Model,R2_CV,MSE_CV
0,LinearRegression,0.9590,133.2762
1,DecisionTreeRegressor,0.9385,200.4229
2,GradientBoostingRegressor,0.9597,130.9626


Ara comparem les mètriques utilitzant **train/test** amb les mètriques utilitzant **cross validation**

In [26]:
pd.concat([metrics_cv, metrics[['R2_Train', 'MSE_Train']] ], axis=1)

,Model,R2_CV,MSE_CV,R2_Train,MSE_Train
0,LinearRegression,0.9590,133.2762,0.9599,133.4369
1,DecisionTreeRegressor,0.9385,200.4229,0.9795,68.1575
2,GradientBoostingRegressor,0.9597,130.9626,0.9614,128.4322


### Comparació Train/Test vs Cross Validation
- En els models de LinearRegression i GradientBoostingRegressor observem que el train/test i el cross validation en quant a R2 tenen uns resultats molt similars. En canvi, en el model de DecisionTreeRegressor, el train/test té un resultat millor en quant a R2.  
- En els models de LinearRegression i GradientBoostingRegressor observem que el train/test i el cross validation en quant a MSE tenen uns resultats molt similars. En canvi, en el model de DecisionTreeRegressor, el train/test té un resultat molt millor en quant a MSE.   

**Conclusió** Per al model **DecisionTreeRegressor és millor aplicar el train/test**. Per a LinearRegression i GradientBoostingRegressor podriem aplicar qualsevol dels 2 perquè s'obtenen resultats similars

## - Exercici 5

Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

Els 4 exercicis anteriors els hem fet utilitzant les 3 variables que tenien una correlació més alta amb el target: DepDelay, TaxiOut, TaxiIn.   
Per realitzar aquest exercici i intentar millorar la predicció, utilitzarem més variables. Compararem els resultats anteriors amb nomès 3 variables amb els resultats afegint més variables i realitzar enginyeria de variables.

In [41]:
airlines1

,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,3,4,1955,WN,-14.0,8.0,IAD,TPA,810,4.0,8.0
1,1,3,4,735,WN,2.0,19.0,IAD,TPA,810,5.0,10.0
2,1,3,4,620,WN,14.0,8.0,IND,BWI,515,3.0,17.0
4,1,3,4,1755,WN,34.0,34.0,IND,BWI,515,3.0,10.0
5,1,3,4,1915,WN,11.0,25.0,IND,JAX,688,4.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1220,DL,25.0,30.0,MSP,ATL,906,9.0,18.0
7009717,12,13,6,600,DL,75.0,57.0,RIC,ATL,481,15.0,34.0
7009718,12,13,6,847,DL,99.0,80.0,ATL,IAH,689,8.0,32.0
7009726,12,13,6,1240,DL,9.0,11.0,IAD,ATL,533,13.0,13.0


### Selecció de variables
No seleccionem les següents variables:
- **CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, SecurityDelay, LateAircraftDelay** ja que expliquen la causa del retard per als vols amb retard >15' de forma exacta. Te la pinta que han omplert quan ja es coneix el retard i s'han quadrat exactament amb el retard.
- **Dest** perque pensem que el destí no té influencia en el target
- **Origin** perquè pensem que la influència en el target serà molt baixa i el cost de processat molt elevat si fem dummies

In [27]:
airlines2 = airlines1.drop(['Dest', 'Origin'], axis=1)

Agafarem una mostra aleatoria del 10% sobre la que treballarem

In [43]:
airlines2 = airlines2.sample(frac=0.1, random_state=0)

In [44]:
airlines2

,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut
6731208,12,1,1,1555,WN,84.0,94.0,276,2.0,8.0
609057,2,3,7,1040,WN,7.0,18.0,697,5.0,7.0
889032,2,2,6,1915,WN,1.0,8.0,197,4.0,10.0
5502092,10,2,4,1845,XE,31.0,35.0,817,5.0,18.0
1699584,3,25,2,1810,AA,62.0,66.0,1103,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...
511846,1,13,7,1915,AA,46.0,35.0,190,15.0,17.0
636696,2,12,2,2105,WN,81.0,56.0,972,5.0,24.0
5079318,9,6,6,1815,US,254.0,268.0,2075,6.0,15.0
51952,1,18,5,1310,WN,92.0,106.0,997,6.0,8.0


### Enginyeria de variables
- Transformarem CRSDepTime en intervals horaris
- Dummificarem **UniqueCarrier i Origin**
- Aplicarem RobustScaler a **DepDelay, Distance, TaxiIn, TaxiOut**, degut a la gran quantitat d'outliers que presenten
- Deixem sense transformar **Month, DayofMonth, DayOfWeek**

In [28]:
airlines2['CRSDepTime'] = airlines2['CRSDepTime'] //100

columns_dummies= ['UniqueCarrier']
columns_robust= ['DepDelay', 'Distance', 'TaxiIn', 'TaxiOut']
#Apliquem RobustScaler
airlines2[columns_robust]= RobustScaler().fit_transform(airlines2[columns_robust])
#Fem dummies
airlines2= pd.get_dummies(airlines2, columns=columns_dummies)

In [29]:
airlines2

,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,UniqueCarrier_9E,UniqueCarrier_AA,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,1,3,4,19,-14.0,-0.390244,0.309560,-0.50,-0.545455,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,3,4,7,2.0,-0.121951,0.309560,-0.25,-0.363636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,3,4,6,14.0,-0.390244,-0.138088,-0.75,0.272727,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,3,4,17,34.0,0.243902,-0.138088,-0.75,-0.363636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,1,3,4,19,11.0,0.024390,0.124431,-0.50,-0.363636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,12,25.0,0.146341,0.455235,0.75,0.363636,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7009717,12,13,6,6,75.0,0.804878,-0.189681,2.25,1.818182,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7009718,12,13,6,8,99.0,1.365854,0.125948,0.50,1.636364,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7009726,12,13,6,12,9.0,-0.317073,-0.110774,1.75,-0.090909,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


Entrenem els mateixos 3 models utilitzant totes les variables una vegada aplicada la enginyeria de variables.

In [30]:
#Fem el train/test
y=airlines2['ArrDelay'].to_numpy()
x=airlines2.drop(['ArrDelay'], axis=1).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

### Model 1: Linear Regression

In [31]:
lrs=LinearRegression().fit(X_train, y_train)
R2_Train = r2_score(y_train, lrs.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, lrs.predict(X_train)).round(4)
R2_Test = r2_score(y_test, lrs.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, lrs.predict(X_test)).round(4)

#Creem un DF amb les mètriques per fer la comparació posterior
data= {'Model':'LinearRegression', 'R2_Train_Sc':R2_Train, 'MSE_Train_Sc':MSE_Train,'R2_Test_Sc':R2_Test, 'MSE_Test_Sc':MSE_Test }
metrics_scaled= pd.DataFrame(data, index=[0])
metrics_scaled

,Model,R2_Train_Sc,MSE_Train_Sc,R2_Test_Sc,MSE_Test_Sc
0,LinearRegression,0.9627,120.4156,0.9627,119.8007


### Model 2.- Decision Tree Regression

In [74]:
dts = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
R2_Train = r2_score(y_train, dts.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, dts.predict(X_train)).round(4)
R2_Test = r2_score(y_test, dts.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, dts.predict(X_test)).round(4)

print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 1.0       R2_Test: 0.9176
MSE_Train: 0.0    MSE_Test: 254.6056


Veiem que el model DecisionTreeRegressor ha produit overfiting. Cambiem els paràmetres del model per evitar que es produeixi

In [32]:
dts = DecisionTreeRegressor(random_state=0, max_depth=10).fit(X_train, y_train)
R2_Train = r2_score(y_train, dts.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, dts.predict(X_train)).round(4)
R2_Test = r2_score(y_test, dts.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, dts.predict(X_test)).round(4)

print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))


R2_Train: 0.9586       R2_Test: 0.9571
MSE_Train: 133.8541    MSE_Test: 137.6794


Veiem que amb el paràmetre **max_depth=10** ja no tenim overfiting i les mètriques estàn amb línia amb les dels altres models

In [33]:
#Afegim les mètriques al DF metrics_scaled
data= {'Model':'DecisionTreeRegressor', 'R2_Train_Sc':R2_Train, 'MSE_Train_Sc':MSE_Train, 'R2_Test_Sc':R2_Test, 'MSE_Test_Sc':MSE_Test }
metrics_scaled = metrics_scaled.append(data, ignore_index=True)

### Model 3: GradientBoostingRegressor

In [34]:
gbrs = GradientBoostingRegressor(random_state=0).fit(X_train, y_train)
R2_Train = r2_score(y_train, gbrs.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, gbrs.predict(X_train)).round(4)
R2_Test = r2_score(y_test, gbrs.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, gbrs.predict(X_test)).round(4)

#Afegim les mètriques al DF metrics_scaled
data= {'Model':'GradientBoostingRegressor', 'R2_Train_Sc':R2_Train, 'MSE_Train_Sc':MSE_Train, 'R2_Test_Sc':R2_Test, 'MSE_Test_Sc':MSE_Test }
metrics_scaled = metrics_scaled.append(data, ignore_index=True)

Veiem el resultat dels 3 models utilitzant les 10 variables i escalant-les

In [35]:
metrics_scaled

,Model,R2_Train_Sc,MSE_Train_Sc,R2_Test_Sc,MSE_Test_Sc
0,LinearRegression,0.9627,120.4156,0.9627,119.8007
1,DecisionTreeRegressor,0.9586,133.8541,0.9571,137.6794
2,GradientBoostingRegressor,0.9623,121.7462,0.9621,121.5798


Ara comparem les mètriques amb els models utilitzant 3 variables (DepDelay, TaxiIn, TaxiOut) amb els models utilitzant 10 variables, escalant algunes i fent dummies amb UniqueCarrier

In [36]:
pd.concat([metrics[['Model','R2_Train', 'MSE_Train']] , metrics_scaled[['R2_Train_Sc', 'MSE_Train_Sc']] ], axis=1)

,Model,R2_Train,MSE_Train,R2_Train_Sc,MSE_Train_Sc
0,LinearRegression,0.9599,133.4369,0.9627,120.4156
1,DecisionTreeRegressor,0.9795,68.1575,0.9586,133.8541
2,GradientBoostingRegressor,0.9614,128.4322,0.9623,121.7462


Observem que utilitzant més variables i escalant-les millorem les mètriques de LinearRegression i GradientBoostingRegressor respecte a utilitzar únicament 3 varibles. Però **la millora és molt baixa** (entre un 0.2% i un 0.5%)

## - Exercici 6

No utilitzis la variable DepDelay a l’hora de fer prediccions

Partim de les dades de l'exercici anterior i eliminem **DepDelay**.    

No utilitzo **CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, SecurityDelay, LateAircraftDelay** ja que per als vols amb retard >15', la seva suma és exactament DepDelay. Si elimino DepDelay a l'hora de fer prediccions i utilitzo unes variables que equivalen a la seva suma, poc em canviarà la predicció...

In [37]:
airlines3=airlines2.drop(['DepDelay'], axis=1)

In [38]:
airlines3

,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,Distance,TaxiIn,TaxiOut,UniqueCarrier_9E,UniqueCarrier_AA,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,1,3,4,19,-14.0,0.309560,-0.50,-0.545455,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,3,4,7,2.0,0.309560,-0.25,-0.363636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,3,4,6,14.0,-0.138088,-0.75,0.272727,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,3,4,17,34.0,-0.138088,-0.75,-0.363636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,1,3,4,19,11.0,0.124431,-0.50,-0.363636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,12,25.0,0.455235,0.75,0.363636,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7009717,12,13,6,6,75.0,-0.189681,2.25,1.818182,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7009718,12,13,6,8,99.0,0.125948,0.50,1.636364,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7009726,12,13,6,12,9.0,-0.110774,1.75,-0.090909,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
#Fem el train/test
y=airlines3['ArrDelay'].to_numpy()
x=airlines3.drop(['ArrDelay'], axis=1).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

### Model 1: Linear Regression

In [49]:
lr6=LinearRegression().fit(X_train, y_train)
R2_Train = r2_score(y_train, lr6.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, lr6.predict(X_train)).round(4)
R2_Test = r2_score(y_test, lr6.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, lr6.predict(X_test)).round(4)

In [50]:
print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 0.114       R2_Test: 0.1162
MSE_Train: 2863.4053    MSE_Test: 2835.0004


### Model 2.- Decision Tree Regression

In [51]:
dt6 = DecisionTreeRegressor(random_state=0, max_depth=10).fit(X_train, y_train)
R2_Train = r2_score(y_train, dt6.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, dt6.predict(X_train)).round(4)
R2_Test = r2_score(y_test, dt6.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, dt6.predict(X_test)).round(4)

In [52]:
print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 0.1299       R2_Test: 0.1149
MSE_Train: 2811.9347    MSE_Test: 2838.9515


### Model 3: GradientBoostingRegressor

In [53]:
gbr6 = GradientBoostingRegressor(random_state=0).fit(X_train, y_train)
R2_Train = r2_score(y_train, gbr6.predict(X_train)).round(4)
MSE_Train= mean_squared_error(y_train, gbr6.predict(X_train)).round(4)
R2_Test = r2_score(y_test, gbr6.predict(X_test)).round(4)
MSE_Test= mean_squared_error(y_test, gbr6.predict(X_test)).round(4)

In [54]:
print('R2_Train: {0}       R2_Test: {1}'.format(R2_Train, R2_Test))
print('MSE_Train: {0}    MSE_Test: {1}'.format(MSE_Train, MSE_Test))

R2_Train: 0.1289       R2_Test: 0.1307
MSE_Train: 2815.2567    MSE_Test: 2788.5447


Amb les variables utilitzades, els 3 models aconsegueixen una explicació del target molt baixa

## Conclusions Finals
Treballem un dataset de vols als USA del 2008 i l'objectiu és trobar un model que expliqui els retards dels vols. Característiques del dataset:   
- Una variable (DepDelay) explica per ella sola el 95% del target
- 5 variables (CarrierDelay, WeatherDelay, NASDelay, SecurityDelay,LateAircraftDelay sumen exactament el valor del target per als vols amb ArrDelay>15'
- Totes les variables, amb excepció de les temporals i les de control, tenen un elevat número de outliers, es a dir, distribucions amb llargues cues.
- La resta de variables presenten correlacions molt baixes amb el target
- No tenim variables climatològiques o d'altre tipus que podrien tenir una elevada influencia en els retards

Plantegem 3 models utilitzant les 3 variables més correlacionades amb el target, diferent estrategies d'entrenament dels models i fem enginyeria de dades. Podem concloure:
- Els 3 models (Linear Regression, Decision Tree Regression, GradientBoostingRegressor) presenten R2 elevades (superiors a 0.96)  i és la **Decission Tree Regression** la que presenta un R2 més elevat del **0.98.**   
- Ajustem els paràmetres dels models amb GridSearch però no aconseguim millores sensibles.
- Entrenem els models amb Train/Test i amb Cross Validation i no obtenim diferències significatives entre les 2 tècniques. En el cas de Decision Tree Regression sembla millor entrenar amb Train/Test

Plantegem els mateixos 3 models utlitzant 10 variables, escalant algunes variables i dumificant una altre. Obtenim:
- Utilitzant més variables i escalant-les millorem totes les mètriques respecte a utilitzar únicament 3 varibles. Però la millora és molt baixa (entre un 0.2% i un 0.5%)

Plantegem els mateixos 3 models utilitzant 9 variables i eliminant DepDelay (variable que explicava el 95% del target) i no utilitzant les variables que sumen DepDelay per a vols amb ArrDelay>15'. Tenim:
- Les correlacions en els 3 models són molt baixes (R2 de 0.13)
- Es fa difícil pensar que un mateix avió que fa un mateix recorregut diari i que presenta un DepDelay promig de pocs minuts, podem trobar un model (amb les variables que tenim) que ens expliqui que alguns dies a l'any el seu retard passi a ser de més de 500' o que expliqui la variabilitat tan alta del target (grans cues)
